<a href="https://colab.research.google.com/github/zile56/RecommendationSystem/blob/main/MovieRecommdendationSystem_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score

In [1]:
import pandas as pd

# Load user ratings
ratings = pd.read_csv("/content/sample_data/u.data", sep="\t", names=["user_id", "item_id", "rating", "timestamp"])

# Load movie titles
movies = pd.read_csv("/content/sample_data/u.item", sep="|", encoding="latin-1", usecols=[0, 1], names=["item_id", "title"])

# Merge both
data = pd.merge(ratings, movies, on="item_id")
data.head()



FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/u.data'

In [ ]:
## creating user-item matrix
user_item_matrix=data.pivot_table(index='user_id',columns='title',values='rating')
user_item_matrix.head()



title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

## user based collaborative filtering
# Fill missing ratings with 0
user_item_matrix_filled = user_item_matrix.fillna(0)

# Compute cosine similarity between users
user_similarity = cosine_similarity(user_item_matrix_filled)

# Convert to DataFrame for easy usage
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)
user_similarity_df.head()



user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.168937,0.048388,0.064561,0.379670,0.429682,0.443097,0.320079,0.078385,0.377733,...,0.372213,0.119860,0.269860,0.193343,0.197949,0.118722,0.315064,0.149086,0.181612,0.399432
2,0.168937,1.000000,0.113393,0.179694,0.073623,0.242106,0.108604,0.104257,0.162470,0.161273,...,0.147095,0.310661,0.363328,0.410725,0.322713,0.231096,0.228793,0.162911,0.175273,0.106732
3,0.048388,0.113393,1.000000,0.349781,0.021592,0.074018,0.067423,0.084419,0.062039,0.066217,...,0.033885,0.043453,0.167140,0.071288,0.126278,0.026758,0.164539,0.102899,0.136757,0.026990
4,0.064561,0.179694,0.349781,1.000000,0.031804,0.068431,0.091507,0.188060,0.101284,0.060859,...,0.054615,0.036784,0.133619,0.196561,0.146058,0.030202,0.196858,0.152041,0.171538,0.058752
5,0.379670,0.073623,0.021592,0.031804,1.000000,0.238636,0.374733,0.248930,0.056847,0.201427,...,0.340183,0.080580,0.095284,0.081053,0.148607,0.071612,0.239955,0.139595,0.153799,0.313941


In [ ]:
## get recommendation for a user ( user based)

def get_user_based_recommendations(user_id, top_n=5):
    # Get similarity scores for this user
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:11]  # Top 10 similar users

    # Get their ratings
    similar_users_ratings = user_item_matrix.loc[similar_users.index]

    # Weighted average recommendation
    weighted_ratings = similar_users_ratings.T.dot(similar_users) / similar_users.sum()

    # Filter movies the user has already rated
    user_rated_movies = user_item_matrix.loc[user_id]
    recommendations = weighted_ratings[~user_rated_movies.notna()]

    return recommendations.sort_values(ascending=False).head(top_n)

get_user_based_recommendations(1)


,0
title,
Stand by Me (1986),3.696740
E.T. the Extra-Terrestrial (1982),3.694561
Speed (1994),3.403344
Mission: Impossible (1996),3.198606
'Til There Was You (1997),NaN


In [ ]:
# Item-based Collaborative Filtering for "Mission: Impossible (1996)"

# Get ratings for the movie
MI_ratings = user_item_matrix["Mission: Impossible (1996)"]

# Correlate all other movies with this one
similar_movies = user_item_matrix.corrwith(MI_ratings)

# Convert to DataFrame
corr_df = pd.DataFrame(similar_movies, columns=["Correlation"])
corr_df.dropna(inplace=True)

# Add number of ratings
ratings_counts = data.groupby('title')['rating'].count()
corr_df['num_of_ratings'] = ratings_counts

# Filter popular movies and sort
recommended = corr_df[corr_df['num_of_ratings'] > 100].sort_values("Correlation", ascending=False)
recommended.head(10)


/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:2922: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:2914: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:2773: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:2773: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:2923: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


,Correlation,num_of_ratings
title,,
Mission: Impossible (1996),1.000000,344
GoldenEye (1995),0.508435,131
Cinderella (1950),0.506632,129
Miracle on 34th Street (1994),0.496293,101
Batman Forever (1995),0.487949,114
"Net, The (1995)",0.482145,120
Pretty Woman (1990),0.479713,164
Young Guns (1988),0.477117,101
"Nutty Professor, The (1996)",0.473758,163


In [ ]:
## Evaluate with Precision@K
def precision_at_k(actual, predicted, k):
    predicted = predicted[:k]  # Top K predictions
    actual_set = set(actual)
    predicted_set = set(predicted)
    hits = len(actual_set & predicted_set)
    return hits / k if k > 0 else 0

# Select user to evaluate
user_id = 1

# Get top 10 predicted movies using your user-based recommender
predicted_movies = get_user_based_recommendations(user_id, top_n=10).index.tolist()

# Get actual liked movies for that user (you can define 'liked' as rating >= 4)
actual_liked = data[(data['user_id'] == user_id) & (data['rating'] >= 4)]['title'].tolist()

# Choose K (e.g. top-5)
k = 5
precision = precision_at_k(actual_liked, predicted_movies, k)
print(f"Precision@{k} for User {user_id}: {precision:.2f}")


Precision@5 for User 1: 0.00
